In [10]:
import pandas as pd
import protfasta
import glob

In [12]:
# Loading in all paths to all existing AD alignments
AD_alignment_filepaths = glob.glob("../data/zoonomia_toga_mca/all_AD_alignments/*")
len(AD_alignment_filepaths)

656

In [14]:
AD_alignment_filepaths[0]

'../data/zoonomia_toga_mca/all_AD_alignments/SRF_406-508.fasta'

In [21]:
known_ADs = pd.read_csv("../output/known_ADs_considering_isoforms_and_canonical.csv")
known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[['Q6UWZ7']],ENST00000321945,Q6UWZ7,DelRosso et al.,CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,NaN
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
739,ZSCAN20,262,341,P17040,"[['P17040', 'P17040-3']]",ENST00000361328,P17040,DelRosso et al.,TF,PSNTSEKEQGPEFWGLSLINSGKRSTADYSLDNEPAQALTWRDSRA...,NaN
740,ZXDA,572,699,P98168,"[['P98168'], ['P98168']]",nan / nan,P98168 / P98168 / P98168,"PMID: 17493635, Soto / R4TA_regions.txt / acti...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
741,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
742,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN


In [24]:
AD_dict = protfasta.read_fasta("../data/zoonomia_toga_mca/all_AD_alignments/ABRAXAS1_121-200.fasta", 
                     invalid_sequence_action = 'ignore')
AD_dict["REFERENCE"]

'LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPLVVANLGMSEQLGYKTVSGSCMSTGFSRAVQTHSS'

In [45]:
known_ADs[known_ADs["Gene"].str.contains("CITED")]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
68,CITED2,220,269,Q99967,[['Q99967']],NaN,Q99967 / Q99967,"MVS fragment / Berlow et al, GSL",TF,TDFIDEEVLMSLVIEMGLDRIKELPELWLGQNEFDFMTDFVCKQQP...,NaN


In [49]:
for path in AD_alignment_filepaths:
    #print(path)
    
    # Extracting info from path name
    gene = path.split("/")[-1].split("_")[0]
    start = int(path.split("/")[-1].split("_")[1].split("-")[0])
    end = int(path.split("/")[-1].split("_")[1].split("-")[1].split(".")[0])

    # AD seq in alignment
    AD_dict = protfasta.read_fasta(path, 
                         invalid_sequence_action = 'ignore')
    alignment_seq = [val for key, val in AD_dict.items() if "REFERENCE" in key][0]
    #print(alignment_seq)
    
    # Getting reference sequence
    df = known_ADs[(known_ADs["Gene"] == gene) &(known_ADs["Start"] == start) & (known_ADs["End"] == end)  ]
    
    if len(df) == 0:
        df = known_ADs[(known_ADs["Start"] == start) & (known_ADs["End"] == end)]
        if len(df) == 0:
            print("ERROR finding " + path)
            print()
        else:
            print("Found " +  path)
            print(df["Gene"].iloc[0])
            print()

    else:
        annot_seq = df["ProteinRegionSeq"].iloc[0]

        if alignment_seq != annot_seq:
            print("MISMATCH " + path.split("/")[-1])
            print(alignment_seq)
            print(annot_seq)
            print()

Found ../data/zoonomia_toga_mca/all_AD_alignments/HIF1A_776-826.fasta
HIF1A / Hif1a / HIF1A

Found ../data/zoonomia_toga_mca/all_AD_alignments/CITED1_145-193.fasta
CITE1/MSG1

Found ../data/zoonomia_toga_mca/all_AD_alignments/LMO2_11-29.fasta
RBTN-2

MISMATCH EBF3_429-551.fasta
NSFSSQLAVNVSETSQANDQVGYSRNTSSVSPRGYVPSSTPQQSNYNTVSTSMNGYGSGAMASLGVPGSPGFLNGSSANSPYGMKQKSAFAPVVRPQASPPPSCTSANGNGLQGLVVPM
NSFSSQLAVNVSETSQANDQVGYSRNTSSVSPRGYVPSSTPQQSNYNTVSTSMNGYGSGAMASLGVPGSPGFLNGSSANSPYGMKQKSAFAPVVRPQASPPPSCTSANGNGLQAMSGLVVPPM

MISMATCH NR1H3_1-96.fasta
MPHSAGGTAGVGLEAAEPTALLTRAEPPSEPTEIRPQKRKKGPAPKMLGNE
MSLWLGAPVPDIPPDSAVELWKPGAQDASSQAQGGSSCILREEARMPHSAGGTAGVGLEAAEPTALLTRAEPPSEPTEIRPQKRKKGPAPKMLGNE

MISMATCH NFATC4_771-902.fasta
TTGCAQPPAVSFLPRPFPSDPYGGRGSSFSLGLPFSPPAPFRPPLPASPPLEGPFPSQSDVHPLPAEGYNKVGPGYGPGEGAPEQEKSRGGYSSGFRDSVPIQGITLEEVGRDLSGAEEPA
TTGCAQPPAVSFLPRPFPSDPYGGRGSSFSLGLPFSPPAPFRPPPLPASPPLEGPFPSQSDVHPLPAEGYNKVGPGYGPGEGAPEQEKSRGGYSSGFRDSVPIQGITLEEVSEIIGRDLSGFPAPPGEEPPA

MISMATCH PRDM1

'LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPLVVANLGMSEQLGYKTVSGSCMSTGFSRAVQTHSS'

In [8]:
known_ADs["ProteinRegionSeq"].iloc[0]

'LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPLVVANLGMSEQLGYKTVSGSCMSTGFSRAVQTHSS'